# 모델 저장하기

- 무엇을 저장해야 할까?
    - 모델의 구조
    - 모델의 weights

## 훈련하는 동안 체크포인트 저장하기

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# 단순한 모델 정의
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

In [ ]:
model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### 체크포인트 콜백 사용하기

In [ ]:
# 체크포인트 파일 경로 설정
checkpoint_path = "training_1/cp.ckpt" 
checkpoint_dir = os.path.dirname(checkpoint_path) # 현재 경로 폴더 반환
checkpoint_dir

'training_1'

In [ ]:
# 모델의 가중치를 저장하는 체크포인트 콜백 선언
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백 전달

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.1914 - accuracy: 0.6741
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 1.1371 - accuracy: 0.6870 - val_loss: 0.7317 - val_accuracy: 0.7590
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4244 - accuracy: 0.8772
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.4135 - accuracy: 0.8840 - val_loss: 0.5549 - val_accuracy: 0.8200
Epoch 3/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2901 - accuracy: 0.9198
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.2847 - accuracy: 0.9220 - val_loss: 0.4998 - val_accuracy: 0.8430
Epoch 4/10
31/32 [============================>.] - ETA: 0s - loss: 0.2042 - accuracy: 0.9506
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - lo

In [ ]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']

 텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트합니다  
 - 아키텍처가 동일한 모델 간에는 가중치를 공유 가능 
 
 - 그렇다면, 원래 모델과 동일한 아키텍처를 가진 모델을 만든 다음 체크포인트 부여해보자.

In [ ]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3697 - accuracy: 0.1250 - 275ms/epoch - 9ms/step
훈련되지 않은 모델의 정확도: 12.50%


허접하다.



In [ ]:
# 체크포인트에서 가중치를 로드하고 평가해보자.
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4097 - accuracy: 0.8620 - 96ms/epoch - 3ms/step
복원된 모델의 정확도: 86.20%


In [ ]:
# 파일 이름에 epoch 값 부여 (format 형식)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

- 체크포인트 주요 파라미터 

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능 ('epoch'=에폭마다, 숫자=해당 배치마다)
    )

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 13ms/step - loss: 1.1606 - accuracy: 0.6670 - val_loss: 0.6935 - val_accuracy: 0.8060
Epoch 2/50
32/32 [==============================] - 0s 14ms/step - loss: 0.4233 - accuracy: 0.8930 - val_loss: 0.5568 - val_accuracy: 0.8300
Epoch 3/50
32/32 [==============================] - 0s 14ms/step - loss: 0.2849 - accuracy: 0.9250 - val_loss: 0.4641 - val_accuracy: 0.8510
Epoch 4/50
32/32 [==============================] - 0s 15ms/step - loss: 0.2029 - accuracy: 0.9510 - val_loss: 0.5627 - val_accuracy: 0.8310
Epoch 5/50
32/32 [==============================] - 0s 14ms/step - loss: 0.1633 - accuracy: 0.9630 - val_loss: 0.4946 - val_accuracy: 0.8460
Epoch 6/50
32/32 [==============================] - 1s 16ms/step - loss: 0.1212 - accuracy: 0.9730 - val_loss: 0.4345 - val_accuracy: 0.8560
Epoch 7/50
32/32 [==============================] - 0s 15ms/step - loss: 0.0955 - accuracy: 0.9860 - val_loss: 0.4288 - val_accuracy: 0.8540
Epoch 8/50
32

In [ ]:
# 동일한 아키텍처를 가진 모델 선언
model = create_model()

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3613 - accuracy: 0.0810 - 386ms/epoch - 12ms/step
훈련되지 않은 모델의 정확도:  8.10%


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0009.ckpt'

In [ ]:
model.load_weights(latest)

# 체크포인트 주지말고, 한번 평가해보자.
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3995 - accuracy: 0.8680 - 91ms/epoch - 3ms/step
훈련되지 않은 모델의 정확도: 86.80%


In [ ]:
### mytraining1 폴더에 저장
### best only로 저장, 최종 값만 저장되도록
### weight값만
checkpoint_path = "mytraining1/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

model = create_model()

model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 15ms/step - loss: 1.1337 - accuracy: 0.6910 - val_loss: 0.7003 - val_accuracy: 0.7880
Epoch 2/50
32/32 [==============================] - 0s 8ms/step - loss: 0.4031 - accuracy: 0.8930 - val_loss: 0.5136 - val_accuracy: 0.8450
Epoch 3/50
32/32 [==============================] - 0s 10ms/step - loss: 0.2916 - accuracy: 0.9310 - val_loss: 0.4707 - val_accuracy: 0.8580
Epoch 4/50
32/32 [==============================] - 0s 11ms/step - loss: 0.2038 - accuracy: 0.9460 - val_loss: 0.4480 - val_accuracy: 0.8590
Epoch 5/50
32/32 [==============================] - 0s 9ms/step - loss: 0.1650 - accuracy: 0.9660 - val_loss: 0.4194 - val_accuracy: 0.8650
Epoch 6/50
32/32 [==============================] - 0s 9ms/step - loss: 0.1200 - accuracy: 0.9780 - val_loss: 0.4026 - val_accuracy: 0.8740
Epoch 7/50
32/32 [==============================] - 0s 11ms/step - loss: 0.0901 - accuracy: 0.9840 - val_loss: 0.4327 - val_accuracy: 0.8630
Epoch 8/50
32/32

In [ ]:
model.load_weights(checkpoint_path)

model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 0.3865 - accuracy: 0.8770 - 90ms/epoch - 3ms/step


[0.38650035858154297, 0.8769999742507935]

In [ ]:
tf.keras.callbacks.ModelCheckpoint?

In [ ]:
### mytraining2 폴더에 저장
### 5epoch마다 저장
### weight값만
checkpoint_path = "mytraining3/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=False, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    period=5, # 체크포인트 저장하는 에폭주기 설정 가능
    save_freq='epoch' # epoch의 끝에서 저장할지 여부
    )

In [ ]:
model = create_model()

model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/50
32/32 [==============================] - 1s 14ms/step - loss: 1.1593 - accuracy: 0.6600 - val_loss: 0.6875 - val_accuracy: 0.7960
Epoch 2/50
32/32 [==============================] - 0s 10ms/step - loss: 0.4204 - accuracy: 0.8860 - val_loss: 0.5142 - val_accuracy: 0.8360
Epoch 3/50
32/32 [==============================] - 0s 9ms/step - loss: 0.2754 - accuracy: 0.9250 - val_loss: 0.4456 - val_accuracy: 0.8680
Epoch 4/50
32/32 [==============================] - 0s 8ms/step - loss: 0.2081 - accuracy: 0.9450 - val_loss: 0.4610 - val_accuracy: 0.8470
Epoch 5/50
32/32 [==============================] - 0s 11ms/step - loss: 0.1539 - accuracy: 0.9690 - val_loss: 0.4447 - val_accuracy: 0.8570
Epoch 6/50
32/32 [==============================] - 0s 11ms/step - loss: 0.1116 - accuracy: 0.9820 - val_loss: 0.4309 - val_accuracy: 0.8570
Epoch 7/50
32/32 [==============================] - 0s 8ms/step - loss: 0.0795 - accuracy: 0.9920 - val_loss: 0.4047 - val_accuracy: 0.8670
Epoch 8/50
32/32

### early stopping & 체크포인트 같이 사용하기

In [ ]:
checkpoint_path = "training_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=True, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images, 
          train_labels,
          epochs=100, 
          callbacks=[cp_callback, early], # 체크포인트 부여, early stopping
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
32/32 [==============================] - 1s 13ms/step - loss: 1.1654 - accuracy: 0.6800 - val_loss: 0.7329 - val_accuracy: 0.7860
Epoch 2/100
32/32 [==============================] - 0s 11ms/step - loss: 0.4138 - accuracy: 0.8850 - val_loss: 0.5519 - val_accuracy: 0.8290
Epoch 3/100
32/32 [==============================] - 0s 11ms/step - loss: 0.2959 - accuracy: 0.9140 - val_loss: 0.4863 - val_accuracy: 0.8470
Epoch 4/100
32/32 [==============================] - 0s 11ms/step - loss: 0.1982 - accuracy: 0.9550 - val_loss: 0.4249 - val_accuracy: 0.8620
Epoch 5/100
32/32 [==============================] - 0s 8ms/step - loss: 0.1531 - accuracy: 0.9700 - val_loss: 0.4279 - val_accuracy: 0.8570
Epoch 6/100
32/32 [==============================] - 0s 9ms/step - loss: 0.1125 - accuracy: 0.9790 - val_loss: 0.4098 - val_accuracy: 0.8680
Epoch 7/100
32/32 [==============================] - 0s 11ms/step - loss: 0.0968 - accuracy: 0.9820 - val_loss: 0.4209 - val_accuracy: 0.8570
Epoch 8/

### 직접 가중치 체크포인트 저장

In [ ]:
# 가중치 저장
model.save_weights('./checkpoints/my_checkpoint.ckpt')

model = create_model()

# 가중치 복원
model.load_weights('./checkpoints/my_checkpoint.ckpt')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4118 - accuracy: 0.8750 - 258ms/epoch - 8ms/step
복원된 모델의 정확도: 87.50%


## 전체 모델 저장(모델레이어와 함께 저장)
1) save_weights_only=False 로 변경  
2) 직접 model.save() : SavedModel(pb)  
3) HDF5 파일로 저장하기

### save_weights_only=False 로 변경

In [ ]:
checkpoint_path = "training_4/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, # 고유한 파일이름 부여 가능 
    monitor='val_loss', # val_loss값을 기준으로 최저점일때를 기록하고 싶다면
    save_best_only=True, # True를 주면 monitor기준 최고상태 기록
    save_weights_only=False, # False면 모델 레이어 및 weight값 모두
    # save_freq=5 # 체크포인트 저장하는 에폭주기 설정 가능
    )

In [ ]:
# 새로운 모델 객체를 만듭니다
model = create_model()

In [ ]:
model.fit(train_images, 
          train_labels,
          epochs=100, 
          callbacks=[cp_callback], # 체크포인트 부여
          validation_data=(test_images,test_labels),
          verbose=1)

Epoch 1/100
30/32 [===========================>..] - ETA: 0s - loss: 1.1939 - accuracy: 0.6594

32/32 [==============================] - 2s 47ms/step - loss: 1.1679 - accuracy: 0.6660 - val_loss: 0.7341 - val_accuracy: 0.7750
Epoch 2/100
32/32 [==============================] - 1s 25ms/step - loss: 0.4300 - accuracy: 0.8790 - val_loss: 0.5525 - val_accuracy: 0.8330
Epoch 3/100
32/32 [==============================] - 1s 28ms/step - loss: 0.2866 - accuracy: 0.9330 - val_loss: 0.4737 - val_accuracy: 0.8430
Epoch 4/100
32/32 [==============================] - 1s 30ms/step - loss: 0.2189 - accuracy: 0.9450 - val_loss: 0.4532 - val_accuracy: 0.8580
Epoch 5/100
32/32 [==============================] - 1s 29ms/step - loss: 0.1566 - accuracy: 0.9640 - val_loss: 0.4179 - val_accuracy: 0.8620
Epoch 6/100
32/32 [==============================] - 0s 10ms/step - loss: 0.1228 - accuracy: 0.9780 - val_loss: 0.4266 - val_accuracy: 0.8630
Epoch 7/100
32/32 [==============================] - 0s 11ms/step - loss: 0.0858 - accuracy: 0.9840 - val_loss: 0.4236 - val_accuracy: 0.8640
Epoch 8/100
32/32 

### SavedModel(pb)
SavedModel 형식은 protobuf 바이너리와 TensorFlow 체크포인트를 포함

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1706 - accuracy: 0.6700
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4176 - accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2784 - accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2154 - accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1660 - accuracy: 0.9640


In [ ]:
# my_model 디렉토리
!ls saved_model

my_model


In [ ]:
# assests 폴더, saved_model.pb, variables 폴더
!ls saved_model/my_model

assets	keras_metadata.pb  saved_model.pb  variables


In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')
#load_model. load_weight가 아님
# 모델 구조를 확인합니다
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


복원된 모델은 원본 모델과 동일한 매개변수로 컴파일되어 있습니다. 이 모델을 평가하고 예측에 사용해 보죠:

In [ ]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4314 - accuracy: 0.8590 - 181ms/epoch - 6ms/step
복원된 모델의 정확도: 85.90%
(1000, 10)


### HDF5 파일로 저장하기
케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다 (모델 레이어까지 저장하는..일종의 확장자)
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1607 - accuracy: 0.6710
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4225 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2956 - accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2079 - accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1621 - accuracy: 0.9680


### h5 VS pb 
- 공통점 : 레이어 & 가중치 모두 저장
- 차이점  
1) HDF5 : 객체 구성을 사용하여 모델 아키텍처를 저장  
2) SavedModel : 실행 그래프(조합에서의 그래프 - 연산구조)를 저장. 원본 코드 없이도 사용자 지정 객체를 저장 가능

#### h5 to pb 변환하기

In [ ]:
!mkdir converted

In [ ]:
model = tf.keras.models.load_model('my_model.h5')

In [ ]:
export_path = 'converted'
model.save(export_path, save_format="tf")